In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx

In [2]:
# First, recreate the EXACT same model architecture
class MeshClassifier(nn.Module):
    def __init__(self, input_dim=3, num_classes=3):
        super(MeshClassifier, self).__init__()

        # filter
        self.filter = nn.Conv1d(input_dim, 32, kernel_size=3, padding=1)

        self.conv1 = nn.Conv1d(32, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.conv4 = nn.Conv1d(256, 512, 1)

        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0)

    def forward(self, vertices_list):
        batch_features = []
        
        for vertices in vertices_list:
            x = vertices.T.unsqueeze(0) # (N, 3) -> (1, 3, N)
            
            x = F.relu(self.filter(x))
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x = F.relu(self.conv3(x))
            x = F.relu(self.conv4(x))
            x = torch.max(x, dim=2)[0]  # Global max pooling (1, 512)
            batch_features.append(x.squeeze(0))  # (512,)

        batch_features = torch.stack(batch_features)  # (batch_size, 512)
        x = F.relu(self.fc1(batch_features))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [3]:
# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MeshClassifier().to(device)
model.load_state_dict(torch.load("model/mesh_classifier.pth"))
model.eval()

MeshClassifier(
  (filter): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv1): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0, inplace=False)
)

In [ ]:
# Test one file

import trimesh

filename = 'chair_124_15.obj'
mesh = trimesh.load(f'obj_files/{filename}')
vertices = torch.FloatTensor(mesh.vertices).to(device)

with torch.no_grad():
    output = model([vertices])  # 取得 logits
    probabilities = F.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

prob_list = [round(p, 4) for p in probabilities.squeeze(0).tolist()]

print(f"Filename: {filename}, Predicted class: {predicted_class}, Probability: {prob_list}")

Filename: chair_124_15.obj, Predicted class: 2, Probability: [0.0018, 0.0, 0.9982]


In [6]:
# Test all files
import os
import trimesh

test_files = "obj_files/"
# results = []

for filename in os.listdir(test_files):
    if filename.endswith(".obj"):
        mesh_path = os.path.join(test_files, filename)

        # 加載 Mesh
        mesh = trimesh.load(mesh_path)
        vertices = torch.FloatTensor(mesh.vertices).unsqueeze(0).to(device)  # (N, 3) -> (1, N, 3)

        with torch.no_grad():
            output = model([vertices.squeeze(0)])  # 取得 logits
            probabilities = F.softmax(output, dim=1)  # 轉換為機率分佈

            if probabilities[0, 2] > 0.3: 
                predicted_class = 2
            else:
                predicted_class = torch.argmax(probabilities, dim=1).item()  # 找出機率最高的類別

        # 轉換為 Python 數值格式，並四捨五入到小數4位
        prob_list = [round(p, 4) for p in probabilities.squeeze(0).tolist()]
        # results.append([filename, predicted_class, prob_list[0], prob_list[1], prob_list[2]])

        print(f"檔案: {filename}, 預測類別: {predicted_class}, 機率: {prob_list}")

檔案: chair_001_01.obj, 預測類別: 0, 機率: [0.9941, 0.005, 0.0009]
檔案: chair_001_02.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_03.obj, 預測類別: 0, 機率: [0.9993, 0.0, 0.0007]
檔案: chair_001_04.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_05.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_06.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_07.obj, 預測類別: 0, 機率: [0.9985, 0.0006, 0.0009]
檔案: chair_001_08.obj, 預測類別: 0, 機率: [0.9382, 0.0225, 0.0393]
檔案: chair_001_09.obj, 預測類別: 0, 機率: [0.9987, 0.0007, 0.0006]
檔案: chair_001_10.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_11.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_12.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_13.obj, 預測類別: 0, 機率: [0.9999, 0.0001, 0.0]
檔案: chair_001_14.obj, 預測類別: 0, 機率: [0.9998, 0.0002, 0.0001]
檔案: chair_001_15.obj, 預測類別: 0, 機率: [0.9984, 0.0011, 0.0005]
檔案: chair_001_16.obj, 預測類別: 0, 機率: [0.9571, 0.0412, 0.0017]
檔案: chair_001_17.obj, 預測類別: 0, 機率: [0.9987, 0.0013, 0.0]
檔案: chair_001_18.obj, 預測類別: 0, 機率: [0.9942, 0.0057, 0